In [ ]:
! cd ../ && python setup.py develop

In [ ]:
import numpy as np
import pandas as pd
import logging
%load_ext autoreload
%autoreload 2

from mercari import nn_conv1d

In [ ]:
small = False
df_train, df_test = nn_conv1d.load_data(small=small)
df_train, df_test, target_scaler, embedding_params = nn_conv1d.preprocess(df_train, df_test)
train_ids, valid_ids, submission = nn_conv1d.load_valid_ids(df_train)

X = nn_conv1d.get_keras_data(df_train)
y = df_train['target']

In [ ]:
train_nonzero = (df_train['price'].iloc[train_ids] != 0).values.nonzero()[0]
X_train = nn_conv1d.slice_keras_data(X, train_ids)
X_train = nn_conv1d.slice_keras_data(X_train, train_nonzero)
y_train = y.values[train_ids][train_nonzero]
X_valid = nn_conv1d.slice_keras_data(X, valid_ids)
y_valid = y[valid_ids]
train_kwargs = dict(epochs=3, batch_size=1024)

In [ ]:
model_rmsles, model_predictions = [], []
for i in range(1):
    model, optimizer = nn_conv1d.get_model(
        X_train, embedding_params, lr=0.004)
    rmsle = nn_conv1d.train_model(
        model, optimizer,
        X_train=X_train, y_train=y_train,
        X_valid=X_valid, y_valid=y_valid,
        target_scaler=target_scaler,
        **train_kwargs)
    model_rmsles.append(rmsle)
    y_pred_valid = model.predict(X_valid, batch_size=nn_conv1d.PRED_BATCH)
    model_predictions.append(y_pred_valid)
    logging.info(f'rmsle: {rmsle:.5f}')
    submission['price'] = nn_conv1d.target_to_price(
        np.mean(model_predictions, axis=0), target_scaler)
    submission.to_csv(f'valid_nn_conv1d_{i}.csv', index=False)
logging.info(
    f'Mean model RMSLE: {np.mean(model_rmsles)} '
    f'± {1.96 * np.std(model_rmsles)}')
mean_rmsle = nn_conv1d.get_rmsle(
    df_train['price'][valid_ids],
    nn_conv1d.target_to_price(np.mean(model_predictions, axis=0),
                              target_scaler))
logging.info(f'RMSLE of prediction mean: {mean_rmsle}')